In [17]:
from langchain_openai import ChatOpenAI
import os

tongyi_chat = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",
    # other params...
)

In [18]:
messages = [
    ("system", "你是一名专业的翻译家，可以将用户的中文翻译为英文。"),
    ("human", "我喜欢编程。"),
]
tongyi_chat.invoke(messages)


AIMessage(content='The English translation of "我喜欢编程" is:\n\n"I like programming."\n\nThis conveys the meaning that you enjoy or have a preference for programming activities. Would you like me to provide any additional context or related expressions?', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 30, 'total_tokens': 73, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-163617e9-bdcc-4a92-9985-2aa952eeb363-0')

In [4]:
for chunk in tongyi_chat.stream(messages):
    print(chunk)

content='I' id='run-71048885-23d5-478f-b2d5-42d73e5b1016'
content=' like' id='run-71048885-23d5-478f-b2d5-42d73e5b1016'
content=' programming' id='run-71048885-23d5-478f-b2d5-42d73e5b1016'
content='.' id='run-71048885-23d5-478f-b2d5-42d73e5b1016'
content='' response_metadata={'finish_reason': 'stop', 'request_id': 'ae82fffd-b7f2-94b0-904f-022ff18441d8', 'token_usage': {'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}} id='run-71048885-23d5-478f-b2d5-42d73e5b1016'


In [5]:
await tongyi_chat.ainvoke(messages)

# stream:
# async for chunk in tongyi_chat.astream(messages):
#    print(chunk)

# batch:
# await tongyi_chat.abatch([messages])


AIMessage(content='I like programming.', response_metadata={'model_name': 'qwen-max', 'finish_reason': 'stop', 'request_id': 'f08b554e-8d1f-913e-93bb-11aced5e1805', 'token_usage': {'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}}, id='run-f61fbf45-036a-44bd-be51-af7567385d8a-0')

In [19]:
from langchain_core.pydantic_v1 import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )

chat_with_tools = tongyi_chat.bind_tools([GetWeather, GetPopulation])
ai_msg = chat_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'call_00a12b6ff8444b72919969'}]

In [20]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    '''Joke to tell user.'''

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_chat = tongyi_chat.with_structured_output(Joke)
structured_chat.invoke("Tell me a joke about cats")


Joke(setup='Why did the cat sit on the computer?', punchline='Because he wanted to keep an eye on the mouse.', rating=None)

In [9]:
ai_msg = tongyi_chat.invoke(messages)
ai_msg.response_metadata

{'model_name': 'qwen-max',
 'finish_reason': 'stop',
 'request_id': 'ab64c178-bac6-950b-9b2d-a5cd44618c85',
 'token_usage': {'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}}